### Librerías

In [1]:
import geopandas as gpd
import pandas as pd
from Valor_catastral_construccion import *
from Valor_catastral_terreno import *
from calculo_predial import *
import random
import numpy as np 


### Base de datos

In [2]:
path_hector = r"G:\Mi unidad\Equipo_Agua\Geo\Data\Prueba\Muestra_modelo_final.xlsx"
# datos = pd.read_csv(path_hector, encoding='utf8')
datos = pd.read_excel(path_hector)
datos.fillna(0,inplace=True)
datos.columns

Index(['Unnamed: 0', 'Indice_gdf', 'index__Cru', 'Tipo calle', 'Nombre Cal',
       'Zona_Banda', 'Manzanas_B', 'COD_BandaV', 'Val m2_Ban', 'Pseudo_cla',
       'Zona_Manz', 'Manzanas_M', 'COD_Manz', 'Tipo', 'Frente', 'Fondo',
       'Area_AH', 'Val m2_Man', 'Pseudo_c_1', 'mun', 'zona', 'manz', 'cve_cat',
       'key', 'Lon', 'Lat', 'Tipo_cruce', 'Tipo_cru_1', 'clase_dete', 'conf',
       'Area_Model', 'Medio', 'Popular', 'Interes', 'Campestre', 'Rural',
       'NivelesM_H', 'NivelesM_1', 'NivelesM_2', 'NivelesP_H', 'NivelesP_1',
       'NivelesIS_', 'NivelesI_1', 'NivelesC_H', 'NivelesR_H', 'Nivel_mini',
       'AlturaM_HM', 'AlturaM__1', 'AlturaM__2', 'AlturaP_HP', 'AlturaP__1',
       'AlturaIS_H', 'AlturaIS_1', 'AlturaC_HC', 'AlturaR_HR', 'Altura_min',
       'Medida_lx', 'Medida_ly', 'geometry', 'Esquinero|Intermedio'],
      dtype='object')

In [3]:
# Obtenemos las columnas de altura
cols_altura = np.array([])
cols_nivel = np.array([])
cols_valor_uni = np.array([])


for col in datos:
    if col.lower().strip().startswith('altura'):
        cols_altura = np.append(cols_altura, col)
    elif col.lower().strip().startswith('nivel'):
        cols_nivel = np.append(cols_nivel, col)
    elif col.lower().strip().startswith('val'):
        cols_valor_uni = np.append(cols_valor_uni,col)
len(cols_altura), len(cols_nivel), len(cols_valor_uni)

(10, 10, 2)

### Llenado de datos faltantes

In [4]:
datos['Frente_base'] = datos['Frente']/100
datos['Fondo_base'] = datos['Fondo']/100
datos['Area_base'] = datos['Area_AH']/1000
datos['posicion'] = np.random.choice(['Interior','Intermedio','Esquinero','Frentes_no_contiguos','Cabecero','Manzanero'], len(datos))
datos['grado_conservacion'] = np.random.choice(['Bueno','Normal','Regular','Malo','Ruinoso'], len(datos))
datos['Area_inscrita'] = datos['Area_Model'].map(lambda x: float(x)*random.uniform(0.65,1))

### Calculo de factores

In [5]:
cols_factor_top = np.array([])
cols_factor_nvl = np.array([])

In [6]:
# Factores catastral del terreno
datos['Factor_posicion']      = datos['Esquinero|Intermedio'].fillna(0).map(lambda x: get_factor_posicion(x))
datos['Terreno_Posicion']     = datos['Factor_posicion'].map(lambda x: 'Intermedio' if x==0.5 else 'otro')
datos['factor_frente']        = pd.Series(map(factor_frente,datos['Medida_lx'],datos['Terreno_Posicion'] ))
datos['factor_fondo']         = pd.Series(map(factor_fondo,datos['Medida_ly'], datos['Fondo_base'],datos['Terreno_Posicion']))
for col in cols_altura:
    nombre_col = 'Factor_Topografia_'+col
    cols_factor_top = np.append(cols_factor_top, nombre_col)
    datos[nombre_col] = pd.Series(map(factor_topografia,datos[col],datos['Medida_ly'],datos['Terreno_Posicion']))

datos['factor_irregularidad'] = pd.Series(map(factor_irregularidad,datos['Area_Model'], datos['Area_inscrita'], datos['Terreno_Posicion']))
datos['factor_area']          = pd.Series(map(factor_area,datos['Area_Model'], datos['Area_base'],datos['Terreno_Posicion']))
datos['factor_restriccion']   = pd.Series(map(factor_restriccion,datos['Area_Model'], datos['Area_Model'],datos['Terreno_Posicion']))

In [7]:
# Factores catastral de la construcción
datos['factor_grado_conservacion'] = pd.Series(map(factor_grado_conservacion,datos['grado_conservacion']))
datos['factor_edad']               = pd.Series(map(get_factor_edad_va,''))
for col in cols_nivel:
    nombre = 'factor_numero_niveles_'+col
    cols_factor_nvl = np.append(cols_factor_nvl, nombre)
    datos[nombre] = pd.Series(map(get_factor_edad_va,datos[col]))

### Calculo de valores catastrales

In [8]:
cols_using = ['Area_Model','factor_frente','factor_fondo', 'factor_irregularidad','factor_area',
              'Factor_posicion','factor_restriccion']
datos.fillna(0,inplace=True)
datos['Val m2_Man'] = datos['Val m2_Man'].str.replace(',','')
for col in cols_using:
    datos[col] = datos[col].astype(float)

![alt text](Data\Valor_cat_terreno.png) 

Formula para obtener el valor catastral del terreno

In [9]:
# Valor catastral del terreno
cols_vc_terr = np.array([])
for col_vu in cols_valor_uni:
    i = 0
    name = col_vu.split('_')[-1]
    for col_ft in cols_factor_top:
        name_col = 'Valor_catastral_terreno_'+name+'_'+str(i)
        datos[name_col] = datos['Area_Model'].astype(float).fillna(0)*datos[col_vu].astype(float).fillna(0)*datos['factor_frente'].astype(float).fillna(0)*datos['factor_fondo'].fillna(0)*datos['factor_irregularidad'].fillna(0)*datos['factor_area'].fillna(0)*datos[col_ft].fillna(0)*datos['Factor_posicion'].fillna(0)*datos['factor_restriccion'].fillna(0)
        i += 1
        cols_vc_terr = np.append(cols_vc_terr,name_col)

![alt text](Data\Valor_cat_const.png) <br>
 Formula para obtener el valor catastral de la construccion

In [10]:
# Valor catastral de la construccion
cols_vc_const = np.array([])
for col_vu in cols_valor_uni:
    i = 0
    name = col_vu.split('_')[-1]
    for col_nv in cols_factor_nvl:
        name_col = 'Valor_catastral_construccion_'+name+'_'+str(i)
        cols_vc_const = np.append(cols_vc_const,name_col)
        datos[name_col] = datos['Area_Model'].astype(float).fillna(0)*datos[col_vu].astype(float).fillna(0)*datos['factor_edad'].astype(float).fillna(0)*datos['factor_grado_conservacion'].astype(float).fillna(0)*datos[col_nv].astype(float).fillna(0)
        i += 1

In [11]:
len(cols_vc_const), len(cols_vc_terr), datos.shape

(20, 20)

![alt text](Data\Valor_cat_final.png)

Formula para obtener el valor catastral final 

(100, 135)

### Exportación de datos

In [17]:
datos.to_csv(r"G:\Mi unidad\Equipo_Agua\Geo\Data\Prueba\Naucalpan/Prueba_aleatoria_100_valor_catastral.csv", 
             index=False, encoding='utf-8')